In [1]:
import findspark
findspark.init()
from pyspark.sql import SparkSession
spark = SparkSession.builder.master('local[1]').appName('pyspark_fresh').getOrCreate()

In [2]:
from pyspark.sql.functions import *
from pyspark.sql.types import *

In [3]:
customers_schema = StructType([
    StructField('Transaction_ID',       IntegerType(), nullable=True),
    StructField('Date',TimestampType(), nullable=True),
    StructField('Customer_Name',    StringType(), nullable=True),
    StructField('Product',    StringType(), nullable=True),
    StructField('Total_Items', IntegerType(), nullable=True),
    StructField('Total_Cost',   DoubleType(), nullable=True),
    StructField('Payment_Method',     StringType(), nullable=True),
    StructField('City',    StringType(), nullable=True),
    StructField('Store_Type',  StringType(), nullable=True),
    StructField('Discount_Applied',  BooleanType(), nullable=True),
    StructField('Customer_Category',  StringType(), nullable=True),
    StructField('Season',  StringType(), nullable=True),
    StructField('Promotion',  StringType(), nullable=True)
])


In [4]:
# D:\pyspark_jupyter_n\data\archive\Retail_Transactions_Dataset.csv
file_loc = 'D:/pyspark_jupyter_n/data/archive/'
file_type = "csv"
# file path
customer_data =   file_loc + 'Retail_Transactions_Dataset.csv'
# file options
first_row_is_header = "True"
delimiter = ","
# create a dataframe using the above details
customers_df = spark.read.format(file_type).option("header", first_row_is_header).schema(customers_schema) .load(customer_data)

In [5]:
# ndf = customers_df.select('Transaction_ID','Product')

In [6]:
pll=[]
for j in customers_df.rdd.collect():
    # print(type(eval(j['Product'])))
    pll.append(eval(j['Product']))

In [7]:
pldf = spark.createDataFrame(pll,schema=ArrayType(StringType()))

In [8]:
pldf.printSchema()

root
 |-- value: array (nullable = true)
 |    |-- element: string (containsNull = true)



In [9]:
pldf = pldf.select(col('value').alias('Product_ar'))

In [10]:
from pyspark.sql.window import Window

In [27]:
pldf=pldf.withColumn('row_index', row_number().over(Window.orderBy(monotonically_increasing_id())))
customers_df=customers_df.withColumn('row_index', row_number().over(Window.orderBy(monotonically_increasing_id())))
new_df = customers_df.join(pldf, on=["row_index"]).drop("row_index")
# new_df.show()

In [28]:
new_df = new_df.drop('Product')

In [33]:
# new_df = new_df.select('*',col('Product_ar').alias('Product')).drop('Product_ar')
new_df = new_df.withColumnRenamed('Product_ar','Product')

In [34]:
new_df.printSchema()

root
 |-- Transaction_ID: integer (nullable = true)
 |-- Date: timestamp (nullable = true)
 |-- Customer_Name: string (nullable = true)
 |-- Total_Items: integer (nullable = true)
 |-- Total_Cost: double (nullable = true)
 |-- Payment_Method: string (nullable = true)
 |-- City: string (nullable = true)
 |-- Store_Type: string (nullable = true)
 |-- Discount_Applied: boolean (nullable = true)
 |-- Customer_Category: string (nullable = true)
 |-- Season: string (nullable = true)
 |-- Promotion: string (nullable = true)
 |-- Product: array (nullable = true)
 |    |-- element: string (containsNull = true)

